In [1]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import gensim.downloader as api

from Linguistic_Features import*
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from textblob import TextBlob

import pandas as pd
import numpy as np
import pickle

unable to import 'smart_open.gcs', disabling that module


In [2]:
def Get_AvgSentence_Embedding(Sentence):
    
    sent_words = word_tokenize(Sentence)
    
    word_embeddings = [] 
    for sent_word in sent_words:
        
        if not sent_word.isalpha():
            continue
        
        if sent_word in vocabulary:
            word_embedding = vectors[sent_word]
            word_embeddings.append(word_embedding)
    
    word_embeddings = np.array(word_embeddings)
    sent_embedding = np.mean(word_embeddings, axis=0)
    
    return sent_embedding

def Load_Trained_Model(ModelPath):
    
    with open(ModelPath, "rb") as file:
        Model_Obj = pickle.load(file)
        
    return Model_Obj

def Task1_Predict(Sentence, Test_Features):
    
    ## Our trained model prediction
    model_predictions = Task1_Model.predict(Test_Features)
    if model_predictions[0] < 0.5:
        model_prediction = 0
    else:
        model_prediction = 1
    
    ## Textblob predictions
    Textblob_prediction = int(TextBlob(Sentence).sentiment.subjectivity)
    
    ## Average Predictions
    Avg_Score = (model_prediction + Textblob_prediction)/2
    
    if Avg_Score < 0.5:
        Avg_Pred = 0
    else:
        Avg_Pred = 1
    
    return Avg_Pred

def Task2_Predict(Sent_Linguistic_Features):
    
    Test_Features = Sent_Linguistic_Features[FeatureList]
    sent_words = list(Sent_Linguistic_Features['word'])
    
    ## Our trained model prediction
    
    ## Naive Bayes
    model_probabilities = Task2_Model1.predict_proba(Test_Features)
    model1_predictions = [model_proba[1] for model_proba in model_probabilities]

    
    ## Random Forest
    model2_predictions = Task2_Model2.predict(Test_Features)
    
    ## Textblob predictions
    Textblob_predictions = []
    for sent_word in sent_words:
        Textblob_predictions.append(TextBlob(sent_word).sentiment.subjectivity)
    
    ## Average Predictions
    Avg_Predictions = []
    
    for i in range(len(sent_words)):
        
        Avg_Score = (model1_predictions[i] + model1_predictions[i] + Textblob_predictions[i])/3
        Pred_Entry = (sent_words[i], Avg_Score)
        Avg_Predictions.append(Pred_Entry)
    
    return Avg_Predictions



In [11]:
def Initialization(Task1Model_Path, Task2Model1_Path, Task2Model2_Path):
    
    global Task1_Model, Task2_Model1, Task2_Model2
    
    Task1_Model = Load_Trained_Model(Task1Model_Path)
    Task2_Model1 = Load_Trained_Model(Task2Model1_Path)
    Task2_Model2 = Load_Trained_Model(Task2Model2_Path)

def Bias_Detection_Suggestions(Sentences, N_Similar=0, TopN_Suggestion=0):
    
    Classification_labels = []
    
    for Sentence in Sentences:
        
        if len(Sentence) < 10:
            continue
        
        print("\n\n")
        print(Sentence)
    
        ## Step 1: Get average word embedding of the sentence
        Sent_Embedding = Get_AvgSentence_Embedding(Sentence)

        ## Step 2: Use best performing task 1 model to predict id sentence is subjective
        Sent_Embedding_Features = pd.DataFrame([Sent_Embedding])
        task1_Prediction = Task1_Predict(Sentence, Sent_Embedding_Features)
        
        ## Step 3: If sentence is objective: Do nothing
        if task1_Prediction==0:
            Classification_label = 0
            print("Subjectivity not identified in the sentence")
        
        ## Step 3: If sentence is subjective: Identify words inducing subjectivity in the sentence
        else:
            Classification_label = 1
            
            sent_words = word_tokenize(Sentence)
            sent_words = [w for w in sent_words if w.isalpha()]
            
            ## Step 3.1: Get liguistic features of the words in the sentence
            Sent_Linguistic_Features = Get_Sent_Linguistic_Features(Sentence)
            
            ## Ignore stopwords
            Sent_Linguistic_Features = Sent_Linguistic_Features[~Sent_Linguistic_Features['word'].isin(stop_words)]
            
            ## Ignore proper nouns (13-->NNP)
            Sent_Linguistic_Features = Sent_Linguistic_Features[Sent_Linguistic_Features['POS']!=13]
            
            ## Step 3.2: Get subjectivity score for all words in the sentence
            Word_Subjectivity_Scores = Task2_Predict(Sent_Linguistic_Features)
            
            ## Step 3.3: Identify word introducing subjectivity in the sentence (max score)
            max_score = 0
            subjective_word = ""
            
            for i in range(len(Word_Subjectivity_Scores)):
                
                current_word = Word_Subjectivity_Scores[i][0]
                subj_score = Word_Subjectivity_Scores[i][1]
                
                if subj_score > max_score and current_word in vocabulary:
                    max_score = subj_score
                    subjective_word = current_word
                    
            if max_score == 0:
                Classification_labels.append(0)
                print("Subjectivity not identified in the sentence")
                continue
                
            print("Sentence is subjective")
            print("Word inducing subjectivity in the sentence: ", subjective_word)
                
            ## Step 3.4: Get alternative words for identified subjective word 
            
            ## Top 10 : words with similar meaning --> words with similar embedding 
            ##          Words most similar to subjective word in order of decresing similarity
            if N_Similar==0:
                N_Similar = 10
            Similar_words = vectors.most_similar(subjective_word, topn=N_Similar)
            Similar_words = [w for w in Similar_words if w[0].isalpha()]
            Similar_words.sort(key = lambda x: x[1], reverse=True)
            
            ## Get least subjective top 3 similar words
    
            Similar_words_features = pd.DataFrame(columns=FeatureList)
            Similar_words_str = []
            
            for similar_word in Similar_words:
                
                word_features = Get_Sent_Linguistic_Features(similar_word[0])
                Similar_words_features = pd.concat([Similar_words_features, word_features])
             
            Subjectivity_scores = Task2_Predict(Similar_words_features)
            Subjectivity_scores.sort(key = lambda x: x[1])
            
            Subjective_word_lemma = lemmatizer.lemmatize(subjective_word)
            Filter_words = [tup for tup in Subjectivity_scores if Subjective_word_lemma not in lemmatizer.lemmatize(tup[0]) and tup[1] < max_score]

            if TopN_Suggestion == 0 or TopN_Suggestion>len(Filter_words):
                TopN_Suggestion = len(Filter_words)
            
            TopN_Suggested_Words = Filter_words[:TopN_Suggestion]
            
            print("Alternate word suggetions: ")
            for suggestion in TopN_Suggested_Words : 
                print(suggestion[0])
            
         
        Classification_labels.append(Classification_label)
    return Classification_labels
        

In [4]:
## Initialization 

Folder_Path = "/Users/pranjali/Downloads/Wiki_BiasDetection"

Vocab_Filepath = Folder_Path + "/Data/Embeddings/pretrained_modified.txt"
Task1Model_Path = Folder_Path + "/Saved_Models/NN/NN_Task1_Trained_Model.pkl"
Task2Model1_Path = Folder_Path + "/Saved_Models/NB/NB_Task2_Trained_Model.pkl"
Task2Model2_Path = Folder_Path + "/Saved_Models/RF/RF_Task2_Trained_Model.pkl"

vectors = KeyedVectors.load_word2vec_format(Vocab_Filepath, binary=False)
vocabulary = vectors.wv.vocab

stop_words = set(stopwords.words('english')) 

lemmatizer = WordNetLemmatizer()

FeatureList = ["POS", "POS_Prev", "POS_Next", "Sent_Position", "Hedge", "Hedge_Context", 
               "Factive", "Factive_Context", "Assertive", "Assertive_Context",
               "Implicative", "Implicative_Context", "Report", "Report_Context", 
                "Entailment", "Entailment_Context", "StrongSub", "StrongSub_Context", 
                "WeakSub", "WeakSub_Context", "Polarity", "Positive", "Positive_Context", 
                "Negative", "Negative_Context", "Bias_Lexicon"]

from Neural_Network import*
Initialization(Task1Model_Path, Task2Model1_Path, Task2Model2_Path)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()
Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator GaussianNB from version 0.22.2.post1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
## Testing

Test_Filepath = Folder_Path + "/Data/Model_Data/FinalTestData.csv"
Test_Data = pd.read_csv(Test_Filepath)
Test_Sentences = Test_Data['sentence']

Classification_labels = Bias_Detection_Suggestions(Test_Sentences, 30, 10)





For a few frequently occurring network motifs we show how they manage noise.
1/1 [==============================] - 0s 23ms/step
Subjectivity not identified in the sentence



and in the dogs , she finds lost pieces of herself as she fumbles with the leash to her own life - struggling to hold on as it pulls her toward a brighter future before her past catches up with her .
1/1 [==============================] - 0s 602us/step
Subjectivity not identified in the sentence



Suggests two approaches to economics of blockchain: innovation-centred and governance-centred.
1/1 [==============================] - 0s 382us/step
Subjectivity not identified in the sentence



being blue is not his only predicament ; he also lacks the ability to outwardly express his emotions .
1/1 [==============================] - 0s 553us/step
Subjectivity not identified in the sentence



on her deathbed , candice klein accidentally asks the question , " what did i ever do to deserve this ? "
1/1 [============

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:88: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Subjectivity not identified in the sentence



It will also show how highly interlinked these disciplines are, and how developments in each drive and influence the other.
1/1 [==============================] - 0s 1ms/step
Subjectivity not identified in the sentence



kok pin is creative and a born artist but his parents would rather he focus on his maths and sciences .
1/1 [==============================] - 0s 899us/step
Sentence is subjective
Word inducing subjectivity in the sentence:  creative
Alternate word suggetions: 
talents
arts
ideas
skills
creativity
innovation
talent
marketing
experience
collaborative



calvin ( cube ) , who inherited the struggling business from his deceased father , views the shop as nothing but a burden and waste of his time .
1/1 [==============================] - 0s 396us/step
Subjectivity not identified in the sentence



This paper examines how the changes in factors of a deep learning network can affect the output.
1/1 [============================

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:88: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Sentence is subjective
Word inducing subjectivity in the sentence:  unforgettable
Alternate word suggetions: 
evocation
revelatory
joyful
evocative
wondrous
glorious
enchanting
uplifting
haunting
heartbreaking



against improbable odds , they race towards a thrilling , climactic confrontation on the mysterious isla de muerta .
1/1 [==============================] - 0s 523us/step
Sentence is subjective
Word inducing subjectivity in the sentence:  thrilling


...

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:88: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:88: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Alternate word suggetions: 
inept
timid
bumbling
feckless
skillful



Senior NFL Writer Jay Glazer is one of NFL's top insiders.
1/1 [==============================] - 0s 368us/step
Subjectivity not identified in the sentence



yet this highly traditional world is facing profound changes due to the postwar influx of foreign images and ideas in japanese culture .
1/1 [==============================] - 0s 440us/step
Subjectivity not identified in the sentence



she wants his blessing to marry a man she loves , but he tells her that she must marry a man of his choosing , and that she must undergo female circumcision .
1/1 [==============================] - 0s 549us/step
Subjectivity not identified in the sentence



for every inspired bit of absurdity , there's a lame piece of improvisational noodling .
1/1 [==============================] - 0s 2ms/step
Sentence is subjective
Word inducing subjectivity in the sentence:  lame
Alternate word suggetions: 
duck
joke
cranky
gimmick
tacky





/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:88: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:88: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Alternate word suggetions: 
years
days
end
time
they



In this paper, we propose a new, fermionic Group Field Theory, posessing a color symmetry, and take the first steps in a systematic study of the topological properties of its graphs.
1/1 [==============================] - 0s 461us/step
Subjectivity not identified in the sentence



rooney in the past is sent to kill sullivan .
1/1 [==============================] - 0s 716us/step
Subjectivity not identified in the sentence



shot on location in the williamsburgh and flatbush neighborhoods of brooklyn , hung-up is a 35mm short story in the tradition of happy gilmore and taxi driver .
1/1 [==============================] - 0s 328us/step
Subjectivity not identified in the sentence



she entranced the founder of the museum of art ( masp ) where the fashion parade was being held , and her career was hence launched .
1/1 [==============================] - 0s 330us/step
Subjectivity not identified in the sentence



watching queen of th

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:88: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [8]:
Classification_labels0 = [l for l in Classification_labels if l==0]
print("Objective: ", len(Classification_labels0))
print("Subjective: ", len(Classification_labels)-len(Classification_labels0))

Objective:  1586
Subjective:  658


In [12]:
## Testing on WikiData

Subjective_Test_Filepath = Folder_Path + "/Data/Task2_FinalData/Filtered_Data.tsv"
Subjective_Test_Data = pd.read_csv(Subjective_Test_Filepath)
Subjective_Test_Sentences = Subjective_Test_Data.iloc[:1500]['Src_Sent']

Ojective_Test_Filepath = Folder_Path + "/Data/Task2_FinalData/WikiEdits_Objective.tsv"
Objective_Test_Data = pd.read_csv(Ojective_Test_Filepath, sep="\t")
Objective_Test_Sentences = Objective_Test_Data.iloc[:500]['Src_Raw']

Test_Sentences = pd.concat([Subjective_Test_Sentences, Objective_Test_Sentences])
Test_Sentences = Test_Sentences.sample(frac=1)

Classification_labels = Bias_Detection_Suggestions(Test_Sentences, 30, 10)





nissan navara is the name for the d22 and d40 generations of nissan pickup truck s sold in asia, australia and europe; in north, central and south america it is sold as the nissan frontier.
1/1 [==============================] - 0s 888us/step
Subjectivity not identified in the sentence



roadrunner records is a record label that was originally known as roadracer records.
1/1 [==============================] - 0s 3ms/step
Subjectivity not identified in the sentence



the first accounts of the crash were conflicting in terms of vehicle speed and posted speed limit, with some of the disparity sourced to a preliminary police investigation, which incorrectly stated that the car was traveling at in a zone and did not attempt to brake.
1/1 [==============================] - 0s 8ms/step
Subjectivity not identified in the sentence



chew stoke is a small , picturesque village within the chew valley about eight miles south of bristol, england.
1/1 [==============================] - 0s 3ms/

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:91: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Sentence is subjective
Word inducing subjectivity in the sentence:  high
Alternate word suggetions: 
levels
school
level
middle
college
lower
well
up
there
at



similar depopulation tactics were adopted by turkey in the 1980s and 1990s to combat the rebellion of the marxist kurdistan workers party in southeastern turkey .
1/1 [==============================] - 0s 444us/step
Subjectivity not identified in the sentence



but the republicans were not in agreement either, and they had to contend with the war in cuba, the islamist terrorists in spanish morocco and the continuance of the carlist wars.
1/1 [==============================] - 0s 322us/step
Subjectivity not identified in the sentence



lucille ball was one of the most popular stars in america during her lifetime and had one of hollywood's longest careers.
1/1 [==============================] - 0s 568us/step


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:91: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Subjectivity not identified in the sentence



since the prc does not recognise the roc as legitimate, prc government or media references to some roc government offices or institutions replace their name by a neutral description which does not imply endorsement of the roc's claim to be a legitimate government of either taiwan or china.
1/1 [==============================] - 0s 2ms/step
Subjectivity not identified in the sentence



and a dollar invested in such programs would not only circulate rapidly but would also have a multiplying effect, sustaining jobs not only among prime contractors but also among their suppliers...substituting accounting discipline for military judgment is not just questionable strategy but incongruous when the obama administration is furiously trying to stimulate the economy.
1/1 [==============================] - 0s 357us/step
Subjectivity not identified in the sentence



the glacier is located in the indian state of jammu and kashmir in the indian sub-con

...

Subjectivity not identified in the sentence



not exactly knowing what to do next, the band decided to keep recording new demos and started searching for a new singer.
1/1 [==============================] - 0s 458us/step
Subjectivity not identified in the sentence



this happens repeatedly with different events in each cycle, each time beginning from a slightly later date.
1/1 [==============================] - 0s 542us/step
Subjectivity not identified in the sentence



bah'u'llh, the prophet-founder of the bah' faith, was a follower of the bb and claimed to be the fulfillment of his prophecy.
1/1 [==============================] - 0s 643us/step
Subjectivity not identified in the sentence



ani difranco wrote a parody of it, called 'tis of thee, which pointed out the flaws of us society, particularly the vast social and economic inqualities.
1/1 [==============================] - 0s 615us/step
Subjectivity not identified in the sentence



this victory occurred on the night fsu ded

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:91: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Sentence is subjective
Word inducing subjectivity in the sentence:  assassinated
Alternate word suggetions: 
assassins
bodyguards
assassination
slain
rabin



some critics felt that his views on the anti-racism movement were closer to the extreme- right than to tradition left-wing politics.
1/1 [==============================] - 0s 445us/step
Subjectivity not identified in the sentence



in more modern terms, stumptown is humorously used as portland's nickname for the city's lack of tall skyscrapers, the highest being only 546 feet.
1/1 [==============================] - 0s 358us/step
Subjectivity not identified in the sentence



also one of the many criticisms of the store is prominently shown -- locking flow team members and janitors overnight in the store.
1/1 [==============================] - 0s 384us/step
Subjectivity not identified in the sentence



the buckley school is a prestigious college preparatory day school, founded in 1933 by isabelle buckley.
1/1 [==================

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:91: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:91: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Alternate word suggetions: 
founders
founder
foundation
formerly
became



to michaels, maus seems to gloss over the racial inequality that has plagued the history of the u.s.
1/1 [==============================] - 0s 486us/step
Subjectivity not identified in the sentence



others question the accuracy or relevance of such tales, and argue that they were generally unheard of before the early 1960s when a strong wave of interest focused on the first clear examples of nessie sightings in the 1930s.
1/1 [==============================] - 0s 468us/step
Subjectivity not identified in the sentence



presentations come in nearly as many forms as there are life situations.
1/1 [==============================] - 0s 428us/step
Subjectivity not identified in the sentence



taken from medina's left side, it shows her standing naked in front of an inconclusive backdrop.
1/1 [==============================] - 0s 716us/step
Subjectivity not identified in the sentence


In [13]:
Classification_labels0 = [l for l in Classification_labels if l==0]
print("Objective: ", len(Classification_labels0))
print("Subjective: ", len(Classification_labels)-len(Classification_labels0))

Objective:  1574
Subjective:  424
